In [1]:
import pandas as pd

import numpy as np

from IPython.display import display

pd.__version__ , np.__version__ #,  matplotlib.__version__, sns.__version__

('0.23.3', '1.14.2')

In [2]:
from sklearn.model_selection import train_test_split, learning_curve, cross_val_score, KFold, StratifiedKFold, \
                                    ShuffleSplit, GridSearchCV, RandomizedSearchCV, cross_val_predict

from sklearn.metrics import roc_auc_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.tree import DecisionTreeClassifier

from sklearn.externals import joblib

In [3]:
pwd

'/Users/DonBunk/Desktop/Google Drive/data_science/Python_Projects/Home_Credit_Default_Risk/final_metafeats_and_predictions/level_3'

In [4]:
cd '/Users/DonBunk/Desktop/Google Drive/data_science/Python_Projects/Home_Credit_Default_Risk/'

/Users/DonBunk/Desktop/Google Drive/data_science/Python_Projects/Home_Credit_Default_Risk


In [5]:
from Home_Credit_package.master_pipeline import master_pipeline

In [6]:
path_to_models = 'saved_models/level_3_models/'

level_3_preds_save_path = 'final_metafeats_and_predictions/level_3/level_3_final_predictions/'

# load this

## load df

In [7]:
# load the training and test data, and merge each with their level_1 metafeatures. 

TRAIN_path = 'wrangling/TRAINING_DATA_create_final_wrangled_csv/'
o_TRAIN_df = pd.read_csv(TRAIN_path + 'complete_initial_wrangled_data.csv', index_col = 'SK_ID_CURR')
TRAIN_L2_mf_path = 'level_2_ensembling/'
TRAIN_level_2_metafeatures_df = pd.read_csv(TRAIN_L2_mf_path + 'FINAL_level_2_meta_features_df.csv', index_col = 'SK_ID_CURR')
TRAIN_df = pd.merge(o_TRAIN_df, TRAIN_level_2_metafeatures_df, left_index=True, right_index=True, how = 'outer' )


TEST_path = 'wrangling/TEST_DATA_create_final_wrangled_csv/'
o_TEST_df = pd.read_csv(TEST_path + 'complete_initial_wrangled_data.csv', index_col = 'SK_ID_CURR')
TEST_L2_mf_path = 'final_metafeats_and_predictions/level_2/level_2_meta_feats/'
TEST_level_2_metafeatures_df = pd.read_csv(TEST_L2_mf_path + 'FINAL_level_2_meta_features_df.csv', index_col = 'SK_ID_CURR')
TEST_df = pd.merge(o_TEST_df, TEST_level_2_metafeatures_df, left_index=True, right_index=True, how = 'outer' )

In [14]:
level_3_predictions = pd.DataFrame(TEST_df.index)
level_3_predictions.set_index('SK_ID_CURR', inplace=True)

# model with only EXT SOURCES + level 2 final scores.

In [9]:
minimal_feats = ['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3',
                 'pwr_rescale_RanFor_EXTpoly_Level2',     
                    'pwr_rescale_LogReg_EXTpoly_Level2',     
                    'pwr_rescale_MLP_EXTpoly_Level2',        
                    'pwr_rescale_RanFor_AllFeats_Level2', 
                    'pwr_rescale_LogReg_AllFeats_Level2',  
                    'pwr_rescale_MLP_AllFeats_Level2',
                    ]

In [10]:
total_df_piped, final_feature_list, total_pipeline, trans_list = master_pipeline(df_in = TRAIN_df[minimal_feats],
                                                                                 int_cutoff=20, 
                                                                                 poly_deg=4, 
                                                                                 feats_with_interaction=[]
                                                                                 )

Total original feat len is 9. Sum of feats is 9.
Final array is length 45. final feature list is length 45.


In [11]:
TEST_piped = total_pipeline.transform(TEST_df)

### AdaBoost + log reg.
Kaggle: Private: 0.76113 Public: 0.76498

In [25]:
my_LgRg_4_Ada = joblib.load(path_to_models + 'my_AdaBC_LogReg_minimal_feats_level_3.joblib')
my_LgRg_4_Ada

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=LogisticRegression(C=1000, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
          learning_rate=1, n_estimators=50, random_state=0)

In [26]:
preds = my_LgRg_4_Ada.predict_proba(TEST_piped)
val_scores = [x[1]for x in preds]

In [27]:
level_3_predictions['my_AdaBC_LogReg_minimal_feats_level_3'] = val_scores

In [28]:
this_model = 'my_AdaBC_LogReg_minimal_feats_level_3'

level_3_predictions[[this_model]].to_csv(level_3_preds_save_path + this_model + '.csv', 
                                         columns = list(level_3_predictions[[this_model]].columns),
                                         header = ['TARGET'],
                                          )

# model with all features

In [29]:
feats_to_drop = ['pwr_rescale_RanFor_EXTpoly_Level2',     
                 'pwr_rescale_LogReg_EXTpoly_Level2',     
                 'pwr_rescale_MLP_EXTpoly_Level2',        
                ]

In [30]:
total_df_piped, final_feature_list, total_pipeline, trans_list = \
master_pipeline(df_in = TRAIN_df.drop(columns = feats_to_drop),
               int_cutoff=20, 
               poly_deg=4, 
               feats_with_interaction=[]
               )

Total original feat len is 88. Sum of feats is 88.
Final array is length 433. final feature list is length 433.


In [31]:
TEST_piped = total_pipeline.transform(TEST_df)

### Adaboost + log reg.
Kaggle: Private: 0.76085 Public: 0.76360

In [32]:
my_AdaBC_LogReg_all_feats = joblib.load(path_to_models + 'my_AdaBC_LogReg_all_feats_level_3.joblib')
my_AdaBC_LogReg_all_feats

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=LogisticRegression(C=200, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
          learning_rate=0.01, n_estimators=50, random_state=0)

In [33]:
preds = my_AdaBC_LogReg_all_feats.predict_proba(TEST_piped)
val_scores = [x[1]for x in preds]

In [34]:
level_3_predictions['my_AdaBC_LogReg_all_feats_level_3'] = val_scores

In [35]:
this_model = 'my_AdaBC_LogReg_all_feats_level_3'

level_3_predictions[[this_model]].to_csv(level_3_preds_save_path + this_model + '.csv', 
                                         columns = list(level_3_predictions[[this_model]].columns),
                                         header = ['TARGET'],
                                          )